In [23]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

### 用户基本信息表

In [8]:
user_info_cloumns = ['user_id', 'sex', 'occupation', 'education', 'marriage', 'household_type']
user_info = pd.read_table("../data/train/user_info_train.txt", names=user_info_cloumns, sep=',')

In [11]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55596 entries, 0 to 55595
Data columns (total 6 columns):
user_id           55596 non-null int64
sex               55596 non-null int64
occupation        55596 non-null int64
education         55596 non-null int64
marriage          55596 non-null int64
household_type    55596 non-null int64
dtypes: int64(6)
memory usage: 2.5 MB


### 银行流水记录  

In [15]:
bank_detail = pd.read_table("../data/train/bank_detail_train.txt", sep=',')

In [18]:
bank_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6070197 entries, 0 to 6070196
Data columns (total 5 columns):
uid         int64
timespan    int64
type        int64
amount      float64
markup      int64
dtypes: float64(1), int64(4)
memory usage: 231.6 MB


In [128]:
bank_detail.rename(columns={'uid':'user_id', 'timespan':'time', 'type':'deal_type', 
                            'amount':'deal_amount', 'markup':'income'}, 
                   inplace=True)

### 用户浏览行为  

In [19]:
browse_history_columns = ['user_id', 'time', 'browse_behavior', 'browse_subbehavior']
browse_history = pd.read_table("../data/train/browse_history_train.txt", names=browse_history_columns, sep=',')

In [21]:
browse_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22919547 entries, 0 to 22919546
Data columns (total 4 columns):
user_id               int64
time                  int64
browse_behavior       int64
browse_subbehavior    int64
dtypes: int64(4)
memory usage: 699.4 MB


### 信用卡账单记录  

In [27]:
bill_detail_columns = ['user_id', 'time', 'bank_id', 'previous_bills', 'previous_repayment', 'creditCardQuota', 
                       'current_balance', 'current_MinRepay', 'consume_times', 'current_bills', 'adjust_money',
                       'revolving_interest', 'available_money', 'cash_Advance_Limit', 'repayment_status']
bill_detail = pd.read_table("../data/train/bill_detail_train.txt", names=bill_detail_columns, sep=',')

In [40]:
bill_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338118 entries, 0 to 2338117
Data columns (total 15 columns):
user_id               int64
time                  int64
bank_id               int64
previous_bills        float64
previous_repayment    float64
creditCardQuota       float64
current_balance       float64
current_MinRepay      float64
consume_times         int64
current_bills         float64
adjust_money          float64
revolving_interest    float64
available_money       float64
cash_Advance_Limit    float64
repayment_status      int64
dtypes: float64(10), int64(5)
memory usage: 267.6 MB


### 放款时间信息

In [140]:
loan_time_columns = ['user_id', 'time']
loan_time = pd.read_table("../data/train/loan_time_train.txt", names=loan_time_columns, sep=',')

### 用户逾期记录   

In [67]:
overdue_columns = ['user_id', 'isOverdue']
overdue = pd.read_table("../data/train/overdue_train.txt", names=overdue_columns, sep=',')

# EDA

### 周期性时间提取

In [ ]:
# import time

# table_names = [bank_detail, browse_history, bill_detail, loan_time]

# for value in table_names:
#     value['month'] = value['time'].apply(lambda x: int(time.strftime("%m", time.localtime(x))))
#     value['day'] = value['time'].apply(lambda x: int(time.strftime("%d", time.localtime(x))))
#     value.drop('time', axis=1, inplace=True)

### 数据筛选

In [141]:
# 合并放款时间信息表与银行流水信息表

bankAndLoan = pd.merge(left=bank_detail, right=loan_time, how='left', on='user_id')

In [142]:
bankAndLoan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6070197 entries, 0 to 6070196
Data columns (total 6 columns):
user_id        int64
time_x         int64
deal_type      int64
deal_amount    float64
income         int64
time_y         int64
dtypes: float64(1), int64(5)
memory usage: 324.2 MB


In [144]:
# 筛选出流水时间 <= 放款时间的记录

temp = bankAndLoan.loc[bankAndLoan['time_x'] <= bankAndLoan['time_y']]

In [149]:
# 去重，筛选用户

unique_id = temp.drop_duplicates(subset=['user_id'], keep='first')[['user_id']]